In [ ]:
# 필요한 라이브러리 로드
import os
import pandas as pd
import xlwings as xw

In [ ]:
# 'adobe_raw' 디렉토리에서 처리할 원본 파일 탐색
# 'New_Weekly_B2B_트래픽_Ken'으로 시작하는 파일 사용
for filename in os.listdir('./adobe_raw/'):
    if filename.startswith('New_Weekly_B2B_트래픽_Ken'):
        b2b_traffic_file = './adobe_raw/' + filename
        break

In [ ]:
# xlwings로 Excel 파일을 열고 데이터 읽기
# DRM 보호 CSV 파일을 읽기 위해 Excel 앱을 직접 제어하는 xlwings 사용
# try...finally 구문으로 Excel 앱의 확실한 종료 보장
workbook = None
try:
    workbook = xw.Book(b2b_traffic_file)
    sheet1 = workbook.sheets[0].used_range.value
    df_b2b_traffic = pd.DataFrame(sheet1)
finally:
    if workbook:
        workbook.app.quit()

In [ ]:
# 데이터 정제 및 컬럼 설정
# 'Week' 문자열이 있는 행을 찾아 그 다음 행부터 데이터로 사용
try:
    begin_index = df_b2b_traffic[df_b2b_traffic[0] == 'Week'].index[0] + 1
    df_b2b = df_b2b_traffic.iloc[begin_index:].reset_index(drop=True).copy()
    df_b2b.columns = ['Date', 'Visit']
except IndexError:
    print("'Week'를 포함하는 행을 찾을 수 없어 처리를 중단합니다.")
    df_b2b = pd.DataFrame() # 빈 데이터프레임 생성

In [ ]:
# 데이터 타입 변환
if not df_b2b.empty:
    df_b2b['Date'] = pd.to_datetime(df_b2b['Date'])
    df_b2b['Visit'] = pd.to_numeric(df_b2b['Visit'], errors='coerce').fillna(0).astype(int)

In [ ]:
# WeekNumber 생성 (ISO 주차 기준)
if not df_b2b.empty:
    df_b2b['ISOYear'] = df_b2b['Date'].dt.isocalendar().year.astype('str')
    df_b2b['ISOWeek'] = df_b2b['Date'].dt.isocalendar().week.astype('str')
    df_b2b['WeekNumber'] = df_b2b['ISOYear'] + '-' + df_b2b['ISOWeek']
    
    # 최종 컬럼 선택 및 정리
    df_b2b = df_b2b[['WeekNumber', 'Visit']]

In [ ]:
# CSV 파일로 저장
if not df_b2b.empty:
    df_b2b.to_csv('./csv_raw/b2b.csv', index=False, encoding='utf-8-sig')
    print("b2b.csv 저장 완료")